In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-3312141a-c5df-3a91-bdc4-37757d6f003c)


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**1) Chang path to darknet**


In [ ]:
%cd /content/drive/My Drive/yolov4/darknet

/content/drive/My Drive/yolov4/darknet


**2) Compile Darknet using Nvidia GPU**

In [ ]:
#change makefile to have GPU and OPENCV enabled
#%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variable ‘it_tb_res’ [-Wunused-variable]
     int it_tb_

In [ ]:
!./darknet detector train data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_last.weights -dont_show -clear 

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.860315, GIOU: 0.857600), Class: 0.998610, Obj: 0.983708, No Obj: 0.020190, .5R: 1.000000, .75R: 1.000000, count: 22, class_loss = 1.084872, iou_loss = 2.174167, total_loss = 3.259038 
 total_bbox = 9034375, rewritten_bbox = 0.054780 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.902414, GIOU: 0.901302), Class: 0.999789, Obj: 0.782252, No Obj: 0.002540, .5R: 1.000000, .75R: 1.000000, count: 6, class_loss = 0.131460, iou_loss = 1.178905, total_loss = 1.310365 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.912610, GIOU: 0.912251), Class: 0.998460, Obj: 0.988631, No

**3) Create train.txt and test.txt**

In [ ]:
import glob
import pandas as pd
import numpy as np
#json_file_pathlist=glob.glob("data/training_dataset/*/*/*.jpg")
json_file_pathlist=glob.glob("data/training_dataset/*/*.jpg")
print(json_file_pathlist)

['data/training_dataset/cabbage/img_17130.jpg', 'data/training_dataset/cabbage/img_17108.jpg', 'data/training_dataset/cabbage/img_17120.jpg', 'data/training_dataset/cabbage/img_17135.jpg', 'data/training_dataset/cabbage/img_17116.jpg', 'data/training_dataset/cabbage/img_17137.jpg', 'data/training_dataset/cabbage/img_170.jpg', 'data/training_dataset/cabbage/img_17114.jpg', 'data/training_dataset/cabbage/img_1711.jpg', 'data/training_dataset/cabbage/img_17119.jpg', 'data/training_dataset/cabbage/img_17129.jpg', 'data/training_dataset/cabbage/img_17107.jpg', 'data/training_dataset/cabbage/img_17126.jpg', 'data/training_dataset/cabbage/img_17109.jpg', 'data/training_dataset/cabbage/img_17123.jpg', 'data/training_dataset/cabbage/img_17111.jpg', 'data/training_dataset/cabbage/img_17138.jpg', 'data/training_dataset/cabbage/img_17128.jpg', 'data/training_dataset/cabbage/img_17112.jpg', 'data/training_dataset/cabbage/img_17136.jpg', 'data/training_dataset/cabbage/img_1710.jpg', 'data/training_d

In [ ]:
len(json_file_pathlist)

2331

In [ ]:
df=pd.DataFrame({"image_path":json_file_pathlist})
df.head()

,image_path
0,data/training_dataset/cabbage/img_17130.jpg
1,data/training_dataset/cabbage/img_17108.jpg
2,data/training_dataset/cabbage/img_17120.jpg
3,data/training_dataset/cabbage/img_17135.jpg
4,data/training_dataset/cabbage/img_17116.jpg


In [ ]:
np.random.seed(5)
shuffle_index=np.random.permutation(df.shape[0])

In [ ]:
training_ratio=int(len(shuffle_index)*0.9)
train_index=shuffle_index[0:training_ratio]
test_index=shuffle_index[training_ratio::]

In [ ]:
training_data=df.iloc[train_index,:]
test_data=df.iloc[test_index,:]

In [ ]:
print(training_data)
print(test_index)

                                             image_path
1418  data/training_dataset/grass_marshroom/img_1518...
1887            data/training_dataset/kiwi/img_2424.jpg
976   data/training_dataset/banana/3577838795_b7edd5...
1125         data/training_dataset/banana/img_60227.jpg
1235          data/training_dataset/banana/img_6060.jpg
...                                                 ...
1261          data/training_dataset/banana/img_6094.jpg
1112         data/training_dataset/banana/img_60254.jpg
1553  data/training_dataset/grass_marshroom/img_1542...
5           data/training_dataset/cabbage/img_17137.jpg
2206  data/training_dataset/wet_marshrrom/img_10221.jpg

[2097 rows x 1 columns]
[ 152 1453  844 1916  174  276 2072  605  694 1802 2118 1800  738 1870
 2293 1808  725  435   83   19 1582  185 1233  333 1255  508 1782 2046
 2122  307 2168  922 1547 1776   17 1743 2115 2172 2153  721 1888 1860
 2224 2282 1126  415 1590 1497  263  393  846  291 1287 1926 1025 1399
  380  967 2013  56

In [ ]:
training_data.to_csv("data/train.txt",index=None,header=None)
test_data.to_csv("data/test.txt",index=None,header=None)

**4) Train model**

In [ ]:
!./darknet detector train data/obj.data cfg/yolo-obj.cfg yolov4.conv.137 -dont_show
#!./darknet detector train data/obj.data cfg/yolo-obj.cfg  yolov4.weights -dont_show


Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.754892, GIOU: 0.745819), Class: 0.989760, Obj: 0.267644, No Obj: 0.006684, .5R: 0.953846, .75R: 0.615385, count: 65, class_loss = 12.403873, iou_loss = 17.894360, total_loss = 30.298233 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.746480, GIOU: 0.739495), Class: 0.994599, Obj: 0.403954, No Obj: 0.017306, .5R: 0.956522, .75R: 0.565217, count: 46, class_loss = 6.732414, iou_loss = 3.564767, total_loss = 10.297181 
 total_bbox = 9410848, rewritten_bbox = 0.054416 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.711538, GIOU: 0.678924), Class: 0.929330, Obj: 0.172939, No Obj: 0.000225, .5R: 0.857143, .75R: 0.571429, count: 7, class_loss = 1.507952, iou_loss = 5.843271, total_loss = 7.351223 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.761080, GIOU: 0.755174), Class: 0.985445, Obj: 0.39119

In [ ]:
!./darknet detector train data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_last.weights -dont_show -clear 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolo-obj
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x 